The Flight.jl package is built upon a hierarchical, causal modeling and
simulation framework. At its core lies the parametric `System` type, which
represents a dynamical system with continuous and/or discrete dynamics. A
`System` may have:
- A continuous state `x`
- A discrete state `s`
- An external input `u`
- An output `y`

### Maybe remove this
The `System`'s continuous dynamics are described by the ordinary differential equation
$$
\dot{x} = g(x, u, s, t)
$$

Its discrete dynamics are described by the difference equation
$$
s_{k+1} = f_d(s_{k}, u_{k}, x_{k}, t_{k})
$$

And its output is given by the algebraic equation
$$
y = h(x, u, s, t)
$$
### Maybe remove this


A `System` is specified through a concrete subtype of the abstract type `SystemDefinition`. This
`SystemDefinition` subtype serves a dual purpose:
1. It defines the values that characterize a specific instance of the `System`. These may
   include other `SystemDefinition` subtypes.
2. It provides a dispatch mechanism for extending the set of functions that define and initialize
   the `System`'s properties (`x`, `u`, `y` and `s`), and those that implement the `System`'s
   dynamics.

To clarify this, let us consider a simple example: a [mass-spring-damper][1] model.

The response of this system is determined by its mass $m$, its spring constant
$k$ and its damping constant $c$. Its continuous dynamics are given by the
differential equation $$ \dot{v} = f - \omega^2_n p - 2\zeta \omega_n v $$ $$
\dot{p} = v $$

Where the following quantities are defined
- Velocity: $v$
- Position: $p$
- Normalized external force: $f = F/m$
- Undamped natural frequency: $\omega_n = \sqrt{{k}/{m}}$
- Damping ratio: $\zeta = {c}/{2m\omega_n}$

Of these, $\omega_n$ and $\zeta$ are constant parameters, $v$ and $p$ make up
the models's continuous state `x`, and $f$ is the external input `u`. As
outputs, we may (arbitrarily) select $p$, $v$ and the acceleration $a =
\dot{v}$. 

In its ordinary version, the mass-spring-damper model has no need for discrete
states. To illustrate a use case for `s`, let's add a twist to the model: we
would like the damper to engage only after the system has been in motion for
more than $\tau_d$ seconds, and disengage again whenever it has been
stationary for more than $\tau_d$ seconds.

This behavior can be implemented by a simple finite state machine that keeps track of three
variables: the damper's state (engaged or disengaged), the last time the system started moving, and
the last time it stopped. These clearly represent states in the model, but unlike $v$ or $p$, their
evolution is not described by a differential equation. Instead, they should be updated at the end of
each integration step. Therefore, they belong in the `System`'s discrete state `s`.

This modification also requires two additional parameters: the damper switching interval $\tau_d$,
and a velocity threshold $v_{\epsilon}$, below which we will consider the system to be stationary.

Now that we have a good idea of what our `System`'s constant parameters should look like, we can
start by writing its `SystemDefinition` as

[1]: https://en.wikipedia.org/wiki/Mass-spring-damper_model

In [1]:
using Flight

@kwdef struct MassSpringDamper <: SystemDefinition
    ω_n::Float64 = 1.0 #undamped natural frequency
    ζ::Float64 = 1.0 #damping ratio
    τ_d::Float64 = 2 #damper enable/disable time interval
    v_ϵ::Float64 = 1e-6 #velocity threshold
end

MassSpringDamper

Now, to define the `System`'s `x`, `u`, `y` and `s`, we must keep a few rules in mind:
1. `x` must be a mutable subtype of `AbstractVector{Float64}` (for example, we
   cannot use a `SVector` from `StaticArrays.jl`)
2. `u` and `s` can be of any type, but they should be mutable or have mutable
   fields so that we can modify them in place
3. `y` can be of any type

Properties `x`, `u` and `s` are preallocated upon initialization and modified in-place during
simulation. In contrast, `y` is meant to be logged, so a new instance of it will be created at every
simulation step. Therefore, to avoid heap allocations and keep our code performant, we should
*really* try to stick to immutable, stack-allocated types.

While `x` can be defined as a plain `Vector{Float64}`, it is usually a good idea
to use a labelled `ComponentVector{Float64}` (provided by the awesome
[ComponentArrays.jl][ca] package).

With all of this in mind, we can define the `System`'s variables through the following type definitions and function extensions:

[ca]: https://github.com/jonniedie/ComponentArrays.jl

In [2]:
using ComponentArrays

@kwdef struct MassSpringDamperY
    p::Float64 = 0.0
    v::Float64 = 0.0
    a::Float64 = 0.0
end

@kwdef mutable struct MassSpringDamperS
    damper_engaged::Bool = false
    t_last_moving::Float64 = 0.0
    t_last_stopped::Float64 = 0.0
end

Systems.X(::MassSpringDamper) = ComponentVector(p = 0.0, v = 0.0)
Systems.U(::MassSpringDamper) = Ref(0.0)
Systems.Y(::MassSpringDamper) = MassSpringDamperY()
Systems.S(::MassSpringDamper) = MassSpringDamperS()

A few comments:
- `X`, `U`, `Y` and `S` are empty, trait-like types, used only for dispatch within the `System` constructor. 
- Our `System`'s input (the mass-normalized force $f$) is a scalar. Since a
  plain `Float64` would be immutable, we wrap it around a `Ref` so we can change
  its value during simulation. Alternatively,
  we could define a custom mutable type holding a single `Float64`.
- Being immutable, we could also declare `Y` to be a `NamedTuple`, but defining a custom type is more convenient in certain scenarios.
- If our `System` had no `x`,`u`, `y` or `s`, their corresponding method definitions could be omitted and they would be initialized to `nothing` by default.

We can now go on to define the `System`'s dynamics. We do so by means of the following function extensions:

In [ ]:
using UnPack

function Systems.f_ode!(sys::System{<:MassSpringDamper})

    @unpack x, u, s, constants = sys
    @unpack p, v = x
    @unpack ω_n, ζ = constants

    f = u[] #de-reference u to get the underlying Float64
    ζ *= s.damper_engaged
    a = f - ω_n^2 * p - 2ζ * ω_n * v

    #update sys.ẋ
    sys.ẋ.v = a
    sys.ẋ.p = v

    #update sys.y (cannot be mutated, we need to assign a new instance to it)
    sys.y = MassSpringDamperY(; p, v, a)
end

function Systems.f_step!(sys::System{<:MassSpringDamper})

    @unpack x, s, t, constants = sys
    @unpack Δt_d, v_ϵ = constants

    #t also needs to be de-referenced
    abs(x.v) > v_ϵ ? s.t_last_moving = t[] : s.t_last_stopped = t[]

    Δt_stopped = t[] - s.t_last_moving
    Δt_moving = t[] - s.t_last_stopped

    if s.damper_engaged
        (Δt_stopped > Δt_d) && (s.damper_engaged = false)
    else
        (Δt_moving > Δt_d) && (s.damper_engaged = true)
    end

end

Systems.f_disc!(::NoScheduling, sys::System{<:MassSpringDamper}) = nothing

Function `f_ode!` packs the `System`'s continuous dynamics $\dot{x} = g(x,u,s,t)$ and observation
equation $y = h(x,u,s,t)$ into a single equation $(\dot{x}, y) = f(x, u, s, t)$. This is a design
decision; for many complex systems in this package, $\dot{x}$ and $y$ share lots of intermediate
computations, which would have to be performed twice were $g$ and $h$ defined in separate methods.

Function `f_step!` implements the `System`'s discrete dynamics $s_{k+1} = f_s(x_{k}, u_{k}, s_{k},
t_{k})$, which is to be executed after each continuous integration step.

A `System` may also have discrete dynamics that update not on every integration step, but with some
arbitrary period $\Delta t$. This is the purpose of function `f_disc!`, which has the signature
above. In this case, it does nothing.

For safety reasons, no trivial fallback methods are provided for these three functions, so they must
be defined for every `System`. For convenience, macros `@no_ode`, `@no_step` and `@no_disc` are
provided. Instead of the trivial `f_disc!` method above, we could have written `@no_disc
MassSpringDamper`.

Add initialization!!!!!!!

In [ ]:
sys_desc = MassSpringDamper(; ω_n = 20.0, ζ = 0.4, τ_d = 1)
sys = System(sys_desc)
@show typeof(sys)
@show propertynames(sys)

typeof(sys) = System{MassSpringDamper, MassSpringDamperY, Base.RefValue{Float64}, ComponentVector{Float64, Vector{Float64}, Tuple{Axis{(p = 1, v = 2)}}}, MassSpringDamperS, @NamedTuple{ω_n::Float64, ζ::Float64, τ_d::Float64, v_ϵ::Float64}, @NamedTuple{}}
propertynames(sys) = (:y, :u, :ẋ, :x, :s, :N, :Δt_root, :t, :n, :constants, :subsystems, :Δt, :ω_n, :ζ, :τ_d, :v_ϵ)


(:y, :u, :ẋ, :x, :s, :N, :Δt_root, :t, :n, :constants, :subsystems, :Δt, :ω_n, :ζ, :τ_d, :v_ϵ)

We can see that all the `System`'s variables have been allocated as specified by our methods, and the parameters for this particular `MassSpringDamper` instance are now available to us in the `System`'s `constants` field. Note the `System`'s time `t` is also stored as a `Ref`.

After defining the `System`'s dynamics, it is usually a good idea to do a quick check for unexpected allocations:

In [6]:
using BenchmarkTools

sys.u[] = 0.0
f_ode!(sys)
@show sys.ẋ
Utils.showfields(sys.y)

sys.u[] = 1.0
f_ode!(sys)
@show sys.ẋ
Utils.showfields(sys.y)

@assert @ballocated(f_ode!($sys)) == 0
@assert @ballocated(f_step!($sys)) == 0


UndefVarError: UndefVarError: `sys` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

Now we are ready to simulate our `System`. For this, we create a `Simulation`:

In [7]:
sim = Simulation(sys);

UndefVarError: UndefVarError: `sys` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

A `Simulation` is a parametric type that couples a `System` with a `OrdinaryDiffEq.ODEIntegrator`. It hooks into the `ODEIntegrator` interface to update the `System`'s internal variables, while adding a bit of machinery to ease user interaction with the `System` during and after simulation. Most of this is done via function wrappers and `DiffEqCallbacks`.

Any ODE algorithm in the `OrdinaryDiffEq` ecosystem can be used for numerical integration. However, fixed-step algorithms are typically preferred for aircraft simulation, because many aircraft systems have discrete dynamics that require a fixed update period (think, for example, of a control law or filter discretized for digital implementation). By default, a `Simulation` chooses a RK4 algorithm. 

`Simulation` extends the `SciMLBase.step!` function, which works like it does for an `ODEIntegrator`. It also defines a `run!` function, which is equivalent to `SciMLBase.solve!`, but allows pacing the integrator for pseudo-real time execution.

The initial values for `x`, `u` and `s` are those held by the `System` when it is passed to the `Simulation` constructor. `Simulation` does not provide a `reinit!` function; the safest and most convenient way to run a simulation more than once is to set up a function that initializes the `System`'s `x`, `u` and `s` to their desired values, and then instantiate a new simulation from it. Of course, if the default values set by the initialization methods are good enough, the `System` can simply be recreated from its descriptor.

In [8]:
sim = Simulation(System(sys_desc); t_end = 10) #instantiate a new simulation using the System's default x, u and s
sim.u[] = 1.0 #we can access the underlying System's fields directly
step!(sim, 5.0, true) #step the simulation to t = 5.0 and stop
sim.u[] = 0.0 #release the external force
step!(sim, 10) #step to the end

UndefVarError: UndefVarError: `sys_desc` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

By default, the `Simulation` logs the `System`'s output `y` at every time step. To retrieve it, we simply do:

In [9]:
ts = TimeSeries(sim);

UndefVarError: UndefVarError: `sim` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

The `TimeSeries` type groups an array of logged values with its corresponding array of simulation times. Here are some examples of what it supports:

In [10]:
@show typeof(ts)
@show length(ts)

t = timestamps(ts)

th_3 = ts[t .> 3]; #can be indexed into
@show length(th_3)

th_p = ts.p #accessing a field of the logged type returns a TimeSeries for that field
@show typeof(th_p)

#default Plot recipes are provided for TimeSeries{<:Real} and TimeSeries{<:AbstractVector{<:Real}}
using Plots
plot(th_p; plot_title = "Mass Position", ylabel = "p (m)") #plot position

UndefVarError: UndefVarError: `ts` not defined in `Main`
Suggestion: check for spelling errors or missing imports.